<a href="https://colab.research.google.com/github/isao1974/ai_calc/blob/master/ai_calc_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://qiita.com/Sa2Knight/items/221be46f2702ae453ba9
を参考に足し算を学習する学習モデルを作成する
モデルは保存する。

In [0]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

formulas = DataFrame([
    [0, 0],
    [0, 1],
    [0, 2],
    [1, 0],
    [1, 1],
    [1, 2],
    [2, 0],
    [2, 1],
    [2, 2]
])
answers = DataFrame([0, 1, 2, 1, 2, 3, 2, 3, 4])

model = linear_model.LinearRegression()
model.fit(formulas, answers)

with open('model.pickle', mode='wb') as fp:
  pickle.dump(model, fp)
  

while True:

     # 標準入力から計算式を取得
     print('> ', end='')
     x, y = list(map(lambda x: int(x), input().split(' ')))

     # 学習モデルを用いて回答を取得し、標準出力
     predected_answer = model.predict([[x, y]])
     print("{0} + {1} = {2}".format(x, y, int(predected_answer[0][0])))

保存したモデルを使う

In [15]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

with open('model.pickle', mode='rb') as fp:
  model = pickle.load(fp)
  
  # 加算
  x, y = 10, 20
  predected_answer = model.predict([[x, y]])
  print("{0} + {1} = {3}".format(x, y, z, int(predected_answer[0])))

10 + 20 = 30


DataFrame の１番目を演算子として学習モデルを作る。
linear_model.LinearRegression だと期待する結果が得られなかったので、
様々な識別器を試してみる。

識別器一覧
https://qiita.com/yhyhyhjp/items/3ca101f1cb5dee310c06

結果は失敗。
原因推測
* 学習データが少ない？
* モデルのハイパーパラメータが調整できてない？
* 機械学習では無理？ディープラーニングだといける？

In [11]:
from pandas import DataFrame
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
h = .02  # step size in the mesh

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]


# １番目は演算子 0:"+", 1:"-"
formulas = DataFrame([
    [0, 0, 0],
    [0, 0, 1],
    [0, 0, 2],
    [0, 1, 0],
    [0, 1, 1],
    [0, 1, 2],
    [0, 2, 0],
    [0, 2, 1],
    [0, 2, 2],
    [1, 0, 0],
    [1, 0, 1],
    [1, 0, 2],
    [1, 1, 0],
    [1, 1, 1],
    [1, 1, 2],
    [1, 2, 0],
    [1, 2, 1],
    [1, 2, 2]
])
answers = DataFrame([0, 1, 2, 1, 2, 3, 2, 3, 4, 0, -1, -2, 1, 0, -1, 2, 1, 0])

for model in classifiers:
  print(model.__class__.__name__)
  model.fit(formulas, answers)

  # 加算
  x, y, z = 0, 10, 20
  predected_answer = model.predict([[x, y, z]])
  print("{0} {1} {2} = {3}".format(x, y, z, int(predected_answer[0])))

  # 減産
  x, y, z = 1, 10, 20
  predected_answer = model.predict([[x, y, z]])
  print("{0} {1} {2} = {3}".format(x, y, z, int(predected_answer[0])))

  

KNeighborsClassifier
0 10 20 = 0
1 10 20 = -1
SVC
0 10 20 = 2
1 10 20 = 2
SVC
0 10 20 = 2
1 10 20 = 2
DecisionTreeClassifier
0 10 20 = 4
1 10 20 = 0
RandomForestClassifier
0 10 20 = 4
1 10 20 = 0
AdaBoostClassifier
0 10 20 = 2
1 10 20 = -2
GaussianNB
0 10 20 = 2
1 10 20 = 2
LinearDiscriminantAnalysis
0 10 20 = 4
1 10 20 = 4
QuadraticDiscriminantAnalysis


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-package

ValueError: ignored